In [3]:
from langchain_core.chat_history import InMemoryChatMessageHistory  
# 메모리에 대화 기록을 저장하는 클래스
# 사용자와 AI가 주고받은 메시지를 리스트 형태로 메모리에 저장
from langchain_core.runnables.history import RunnableWithMessageHistory  
# 메시지 기록을 활용해 실행 가능한 래퍼 클래스
# LLM 모델을 감싸서, 이전 대화 기록을 자동을 불러오고 새 기록을 자동 저장해줌
from langchain_openai import ChatOpenAI  
# 오픈AI 모델을 사용하는 랭체인 챗봇 클래스
from langchain_core.messages import HumanMessage
# 랭체인에서 "사용자의 입력 메시지"를 표현하는 객체입니다. 
# LLM에게는 사람이 한 말을 이렇게 포장해서 전달합니다. 

model = ChatOpenAI(model="gpt-4o-mini")
# 예를 들어 model.invoke([HumanMessage("안녕?")]) 하면,
# 이전 대화 기록 없이 단순히 한 문장만 보고 답합니다.

store = {}
# 세션별 대화 기록을 저장할 딕셔너리
# store = {"abc2": InMemoryChatMessageHistory(),  # 이성용 세션"guest1": InMemoryChatMessageHistory(),  # 다른 사용자 세션}
# 즉, “사용자 이름표(session_id)”를 붙여서 따로따로 기억합니다.

def get_session_history(session_id: str):
    # 세션 ID에 따라 대화 기록을 가져오는 함수
    # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()  # 메모리에 대화 기록을 저장하는 객체 생성
    return store[session_id]  # 해당 세션의 대화 기록을 반환

# ✅ 단계별 설명:
# 1️⃣ session_id(예: "abc2")로 대화 기록을 찾습니다.
# 2️⃣ 만약 아직 없으면 새로 InMemoryChatMessageHistory()를 만들어 저장합니다.
# 3️⃣ 있으면 기존 것을 그대로 반환합니다.

with_message_history = RunnableWithMessageHistory(model, get_session_history)
# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성

In [4]:
config = {"configurable": {"session_id": "abc2"}}
# config는 LangChain이 실행 옵션을 받을 때 사용하는 표준 형식
# 그중 "configurable" 키는 “사용자가 직접 바꿀 수 있는 설정값들”을 담는 공간
# "session_id": "abc2" → 이 실행이 "abc2"라는 세션(사용자)의 대화임을 뜻합니다.
# 이걸 LangChain이 읽어서 자동으로 get_session_history("abc2")를 호출합니다.

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 이성용이야.")],
    config=config,
)


print(response.content)

안녕, 이성용님! 만나서 반가워요. 어떻게 도와드릴까요?


In [5]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

당신의 이름은 이성용입니다! 다른 질문이나 이야기하고 싶은 것이 있으면 말씀해 주세요.


In [7]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

response.content

'죄송하지만, 당신의 이름을 알 수 있는 정보가 없습니다. 당신의 이름이 궁금하다면 직접 말씀해 주시면 고맙겠습니다!'

In [8]:
config = {"configurable" : {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
    config=config,
)

response.content

'우리는 당신의 이름이 이성용이라는 것에 대해 이야기했어요. 혹시 더 궁금한 점이 있거나 다른 주제로 이야기하고 싶으신가요?'

In [9]:
config = {"configurable": {"session_id": "abc2"}}
for r in with_message_history.stream(
    [HumanMessage(content = "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")],
    config=config,
):
    print(r.content, end="|")

|이|성|용|이라는| 이름|은| 한국|에서| 흔|히| 사용|되는| 이름|이|니|,| 한국| 사람|일| 가능|성이| 높|습니다|.| 

|한국|의| 문화|에| 대해| 간|단|히| 말씀|드|리|면|:

|1|.| **|음|식|**|:| 한국| 음식|은| 다양|하고| 풍|부|하며|,| 김|치|,| 비|빔|밥|,| 불|고|기| 등이| 유명|합니다|.| 전|통|적으로| 한|식|은| 밥|과| 국|,| 여러| 가지| 반|찬|으로| 구성|됩니다|.

|2|.| **|전|통| 의|상|**|:| 한|복|은| 한국|의| 전|통| 의|상|으로|,| 특별|한| 날|이나| 명|절|에| 입|습니다|.| 화|려|한| 색|상|과| 독|특|한| 디자인|이| 특징|입니다|.

|3|.| **|축|제|**|:| 한국|에는| 다양한| 전|통| 축|제가| 있습니다|.| 예|를| 들어|,| 설|날|(|새|해|)|과| 추|석|(|가|을| 수|확| 축|제|)은| 가족|과| 함께| 모|여| 음|식을| 나|눠| 먹|고|,| 전|통|놀이|를| 즐|기는| 시|기|입니다|.

|4|.| **|예|술|**|:| 한국|은| 음악|,| 춤|,| 문|학|,| 미|술| 등| 다양한| 분야|에서| 풍|부|한| 예|술|적| 유|산|을| 가지고| 있습니다|.| K|-P|op|과| 영화|가| 세계|적으로| 유명|해|지|면서| 현대| 한국| 문화|가| 국제|적으로| 크게| 영향을| 미|치|고| 있습니다|.

|궁|금|한| 점|이| 더| 있|거나| 다른| 주|제|에| 대해| 알고| 싶|으면| 말씀|해| 주세요|!||||